Data Source : https://www.kaggle.com/tongpython/cat-and-dog

Dikarenakan masih kurangnya pemahaman saya, saya mengikuti tutorial dari [site ini]('https://becominghuman.ai/building-an-image-classifier-using-deep-learning-in-python-totally-from-a-beginners-perspective-be8dbaf22dd8')

In [ ]:
!mkdir ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download tongpython/cat-and-dog

In [ ]:
import zipfile, os
local_zip = '/content/cat-and-dog.zip'

# Buat dataset folder
zip_target = '/content/data'
!mkdir /content/data/

# Unzip dataset
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall(zip_target)
zip_ref.close()

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
import numpy as np

In [ ]:
classifier = Sequential()

classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(Flatten())
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

In [ ]:
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
train_data = '/content/data/training_set/training_set'
test_data = '/content/data/test_set/test_set'

train_datagen = ImageDataGenerator(rescale          = 1./255, 
                                   shear_range      = 0.2, 
                                   zoom_range       = 0.2, 
                                   horizontal_flip  = True
                                   )

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory(train_data, 
                                                 target_size  = (64,64),
                                                 batch_size   = 32,
                                                 class_mode   = 'binary'
                                                 )

test_set = test_datagen.flow_from_directory(test_data,
                                            target_size=(64,64),
                                            batch_size=32,
                                            class_mode='binary'
                                            )

In [ ]:
classifier.fit(training_set,
              steps_per_epoch  = 249,
              epochs           = 25,
              validation_data  = test_set,
              validation_steps = 62
              )

In [ ]:
# test_image = image.load_img('/content/guess1.jpg',
#                             target_size=(64,64)
#                             )
test_image = image.load_img('/content/guess4.jpg',
                            target_size=(64,64)
                            )
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
training_set.class_indices

if result[0][0]==1:
  prediction = 'Dog'
else:
  prediction = 'Cat'

print(prediction)

In [ ]:
classifier.save('/content/model.h5')